In [1]:
!pip install pandas pandas-ta

In [2]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

from utils import load_dataset

In [3]:
history = pd.read_csv('./NFLX.csv')

In [4]:
# Reverse dataframe to sort prices by date, ascending order
history = history.iloc[::-1]
history = history.reset_index(drop=True)

In [5]:
history

,Date,Close,Volume,Open,High,Low
0,08/16/2013,36.9814,21354409,36.1728,37.1643,35.8758
1,08/19/2013,37.1114,15494785,36.8571,37.6428,36.8214
2,08/20/2013,39.0414,30108269,37.5614,39.0771,37.5000
3,08/21/2013,38.6243,21240197,38.9843,39.2357,38.2143
4,08/22/2013,38.5357,9961507,38.9314,39.0657,38.3186
...,...,...,...,...,...,...
2510,08/08/2023,438.3000,3767195,440.3600,443.5700,435.1000
2511,08/09/2023,428.9000,3846639,439.7400,440.6200,427.5900
2512,08/10/2023,429.9800,5467607,431.1200,438.1200,428.6900
2513,08/11/2023,421.6600,3927914,428.9800,430.6317,421.3277


In [6]:
frame_window = 10
last_history_included = frame_window - 1

# Initial window/preliminary data
df = history.iloc[0:frame_window]

In [7]:
def compute_indicators(df):
    df = df.copy()
    
    def get_sma_indicator(length):
        values = ta.sma(df["Close"], length=length)
        if values is None:
            return -1
        return values.where(np.isfinite, -1)
    
    df.loc[:, "MA20"] = get_sma_indicator(length=20)
    df.loc[:, "MA50"] = get_sma_indicator(length=50)
    df.loc[:, "MA100"] = get_sma_indicator(length=100)
    
    return df

def update_and_recalculate(df, series_index):
    # Get new update to include in frame
    new_data = history.iloc[series_index].copy()

    # Insert row in frame
    df.loc[len(df)] = new_data

    # Compute TA
    return compute_indicators(df)

In [8]:
df = compute_indicators(df)

In [9]:
df

,Date,Close,Volume,Open,High,Low,MA20,MA50,MA100
0,08/16/2013,36.9814,21354409,36.1728,37.1643,35.8758,-1,-1,-1
1,08/19/2013,37.1114,15494785,36.8571,37.6428,36.8214,-1,-1,-1
2,08/20/2013,39.0414,30108269,37.5614,39.0771,37.5000,-1,-1,-1
3,08/21/2013,38.6243,21240197,38.9843,39.2357,38.2143,-1,-1,-1
4,08/22/2013,38.5357,9961507,38.9314,39.0657,38.3186,-1,-1,-1
5,08/23/2013,39.7657,22322384,38.5786,39.7700,38.4314,-1,-1,-1
6,08/26/2013,40.3886,30688779,39.4443,41.4206,39.1486,-1,-1,-1
7,08/27/2013,39.4343,23054850,39.8786,40.8400,39.2143,-1,-1,-1
8,08/28/2013,40.4800,19354964,39.8086,40.7143,39.4857,-1,-1,-1
9,08/29/2013,41.1214,19059788,40.5757,41.4614,40.5486,-1,-1,-1


In [10]:
window = df.loc[0:frame_window-1,['Date','Close', 'MA20', 'MA50', 'MA100']].to_numpy()
window

array([['08/16/2013', 36.9814, -1, -1, -1],
       ['08/19/2013', 37.1114, -1, -1, -1],
       ['08/20/2013', 39.0414, -1, -1, -1],
       ['08/21/2013', 38.6243, -1, -1, -1],
       ['08/22/2013', 38.5357, -1, -1, -1],
       ['08/23/2013', 39.7657, -1, -1, -1],
       ['08/26/2013', 40.3886, -1, -1, -1],
       ['08/27/2013', 39.4343, -1, -1, -1],
       ['08/28/2013', 40.48, -1, -1, -1],
       ['08/29/2013', 41.1214, -1, -1, -1]], dtype=object)

In [11]:
from bot import Bot

# Start iterating over the history data. simulates that we gather price data from an external provider
# at every new step/iteration, we add the price data into the dataframe df and recompute TAs
# then we move the window 1 day forward and only pass into the bot that window
b = Bot()
start_index = 0
while last_history_included < 110:
    window = df.loc[start_index:frame_window+start_index-1,['Date','Close', 'MA20', 'MA50', 'MA100']].to_numpy()
    b.think(window)
    
    last_history_included += 1
    start_index += 1
    df = update_and_recalculate(df, last_history_included)

Date: 08/16/2013, Price: 36.9814, MA20: -1, MA50: -1, MA100: -1

Date: 08/19/2013, Price: 37.1114, MA20: -1, MA50: -1, MA100: -1

Date: 08/20/2013, Price: 39.0414, MA20: -1, MA50: -1, MA100: -1

Date: 08/21/2013, Price: 38.6243, MA20: -1, MA50: -1, MA100: -1

Date: 08/22/2013, Price: 38.5357, MA20: -1, MA50: -1, MA100: -1

Date: 08/23/2013, Price: 39.7657, MA20: -1, MA50: -1, MA100: -1

Date: 08/26/2013, Price: 40.3886, MA20: -1, MA50: -1, MA100: -1

Date: 08/27/2013, Price: 39.4343, MA20: -1, MA50: -1, MA100: -1

Date: 08/28/2013, Price: 40.48, MA20: -1, MA50: -1, MA100: -1

Date: 08/29/2013, Price: 41.1214, MA20: -1, MA50: -1, MA100: -1

Date: 08/19/2013, Price: 37.1114, MA20: -1.0, MA50: -1.0, MA100: -1.0

Date: 08/20/2013, Price: 39.0414, MA20: -1.0, MA50: -1.0, MA100: -1.0

Date: 08/21/2013, Price: 38.6243, MA20: -1.0, MA50: -1.0, MA100: -1.0

Date: 08/22/2013, Price: 38.5357, MA20: -1.0, MA50: -1.0, MA100: -1.0

Date: 08/23/2013, Price: 39.7657, MA20: -1.0, MA50: -1.0, MA100: -1.

In [12]:
df

,Date,Close,Volume,Open,High,Low,MA20,MA50,MA100
0,08/16/2013,36.9814,21354409,36.1728,37.1643,35.8758,-1.00000,-1.000000,-1.000000
1,08/19/2013,37.1114,15494785,36.8571,37.6428,36.8214,-1.00000,-1.000000,-1.000000
2,08/20/2013,39.0414,30108269,37.5614,39.0771,37.5000,-1.00000,-1.000000,-1.000000
3,08/21/2013,38.6243,21240197,38.9843,39.2357,38.2143,-1.00000,-1.000000,-1.000000
4,08/22/2013,38.5357,9961507,38.9314,39.0657,38.3186,-1.00000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...
106,01/17/2014,47.1486,16415985,47.4386,47.8314,46.8914,50.66434,50.504024,47.549796
107,01/21/2014,46.9586,25210193,47.4614,47.4961,45.7500,50.32127,50.484254,47.625039
108,01/22/2014,47.6757,48198577,47.0714,47.7770,46.7644,50.02170,50.503882,47.696996
109,01/23/2014,55.5314,91278487,55.3428,56.5186,53.9271,50.07984,50.657654,47.841096
